In [15]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from sklearn.metrics import pairwise_distances
from scanpy import read_h5ad
from pathlib import Path
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd

#distribution of distances between 2 types instead of just observation frequency
cell_h5ad = read_h5ad('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/resegmented_data_copies/resegmented_121222_annotated_with_depth.h5ad')
#obs has class, obsm spatial has location. IDs are matched (so spatial 0 = obs 0)
cell_data = cell_h5ad.obs[cell_h5ad.obs['selected_cells']==True] #using napari_x and napari_y for now
# cell_data[['x', 'y']] = cell_h5ad.obsm['spatial']

/tmp/ipykernel_3405285/2654149373.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [16]:
#subset_data['subclass_scANVI'].unique()
barcode = '1218552479'
#create all combinations of subclasses (for a single dataset, commend out next line for all)
subset_data= cell_data[cell_data['filename']==barcode].copy() 
#cut down to even smaller subset of just subclass, x, y.
subclasses = list(subset_data.groupby('subclass_scANVI').groups.keys())
unique_subset_pairs = list(itertools.combinations(subclasses, 2))
root_folder = Path('/home/imaging_mfish/surveyNAS05/scratch/human/AD_MTG/results/manuscript_cci/')
root_folder.joinpath(barcode).mkdir(parents=True, exist_ok=True)

In [18]:
import sys
# print(unique_subset_pairs)
#iterate over pairs, then over circle point multipliers to get all possible pairwise distance groupings
#save to a separate file each time
obs_freqs = []
for class_pair in unique_subset_pairs:
    #for each pair of subclasses in all possible pairs of subclasses, skip if we've seen it before (i.e., don't run x vs y if we've run y vs x).
    #additionally, itertools.combinations doesn't return x vs x or y vs y so add those to the potential iteration options if they haven't been done before.
    pair_iters = [[class_pair[0], class_pair[1]]]
    if class_pair[0] not in obs_freqs:
        pair_iters.append([class_pair[0], class_pair[0]])
        obs_freqs.append(class_pair[0])
    if class_pair[1] not in obs_freqs:
        pair_iters.append([class_pair[1], class_pair[1]])
        obs_freqs.append(class_pair[1])
    for group in pair_iters:
        print(group)
        #create blank df for holding the one calculation of true distances and 10 calculations of randomly shifted distances
        temp_df = pd.DataFrame()
        group_name = str(group[0])+','+str(group[1])
        for i in range(0,11): #11
            print('tempdf: ', sys.getsizeof(temp_df)/1000000000)
            if i == 0:
                pair_0 = cell_data[cell_data['subclass_scANVI']==group[0]][['napari_x', 'napari_y']].values.copy()
                pair_1 = cell_data[cell_data['subclass_scANVI']==group[1]][['napari_x', 'napari_y']].values.copy() #is this 'to_numpy' legit? is this the issue? otherwise .values()
                index = '_realdata'
            else:
                # uniformly random values within 50um circle
                n = max(len(cell_data[cell_data['subclass_scANVI']==group[0]]), len(cell_data[cell_data['subclass_scANVI']==group[1]]))
                radius = np.random.uniform(low=0, high=50, size=n*5)  # varies within 50
                theta = np.random.uniform(low=0, high=2*np.pi, size=n*5)  # angle
                rand_circle_points = pd.DataFrame({'x': np.sqrt(radius) * np.cos(theta), 'y':np.sqrt(radius) * np.sin(theta)}) #circle
                rand_subset_0 = rand_circle_points.sample(len(cell_data[cell_data['subclass_scANVI']==group[0]])).values.copy()
                rand_subset_1 = rand_circle_points.sample(len(cell_data[cell_data['subclass_scANVI']==group[1]])).values.copy()
                pair_0 = cell_data[cell_data['subclass_scANVI']==group[0]][['napari_x', 'napari_y']].values.copy()+rand_subset_0
                pair_1 = cell_data[cell_data['subclass_scANVI']==group[1]][['napari_x', 'napari_y']].values.copy()+rand_subset_1
                index = '_iter'+str(i)
            print('pair sizes: ', sys.getsizeof(pair_0), sys.getsizeof(pair_1))
            clust_dists_pair = np.tril(pairwise_distances(pair_0, pair_1, n_jobs=1)) #size = 3.9gb?! oh no that's small, size is freaking 15gb when doing astro x astro. 
            print('clust dists sizes: ', sys.getsizeof(clust_dists_pair))
            clust_dists_pair[clust_dists_pair==0] = np.nan
            pair_col = group_name+index
            pair_df = pd.DataFrame(pd.Series(clust_dists_pair.flatten()[:]), columns = [pair_col])
            temp_df = pd.concat([temp_df, pair_df], axis=1) #this might be another part of the problem
        if '/' in group_name:
            group_name = group_name.replace('/', '')
        csv_name = str(root_folder.joinpath(barcode))+'/'+barcode+'_'+group_name+'_10423.csv'
        #takes a long time? try pyarrow, but also its just big!
        temp_df.to_csv(csv_name, index=False)

['Astro', 'Chandelier']
tempdf:  1.6e-08
pair sizes:  696840 18088


/tmp/ipykernel_3405285/1979236002.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  print('tempdf: ', sys.getsizeof(temp_df)/1000000000)


clust dists sizes:  391208400
tempdf:  0.391208424
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  0.782416704
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  1.173624984
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  1.564833264
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  1.956041544
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  2.347249824
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  2.738458104
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  3.129666384
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  3.520874664
pair sizes:  696840 18088
clust dists sizes:  391208400
tempdf:  3.912082944
pair sizes:  696840 18088
clust dists sizes:  391208400
['Astro', 'Astro']
tempdf:  1.6e-08
pair sizes:  696840 696840


/tmp/ipykernel_3405285/1979236002.py:22: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  print('tempdf: ', sys.getsizeof(temp_df)/1000000000)


clust dists sizes:  15169336320
tempdf:  15.169336348
pair sizes:  696840 696840


KeyboardInterrupt: 

In [4]:
new_root = root_folder.joinpath(barcode)
cci_csvs = new_root.glob('*.csv')
obs_freqs = pd.DataFrame(columns=['pair_0', 'pair_1', 'z_score'])
for cci in list(cci_csvs):
    # calculate one-tailed z-test of observed vs randomized for every group
    group_df = pd.read_csv(cci)
    slim_df = group_df[group_df.transform(lambda x: x <= 15)].dropna(how='all')
    display(slim_df.head())
    sample_colname = [x for x in list(slim_df.columns) if '_realdata' in x][0]
    sample = slim_df.loc[:, slim_df.columns == sample_colname].count()
    population = slim_df.loc[:, slim_df.columns != sample_colname].count()
    # z = (sample mean – population mean) / [population standard deviation/sqrt(n)]
    z = (sample - np.mean(population)) / (np.std(population)/np.sqrt(10))
    comp_vals = sample_colname.split(',')
    print(z.values[0])
    obs_freqs.loc[len(obs_freqs), obs_freqs.columns] = comp_vals[0], comp_vals[1].split('_')[0], z.values[0]
    if z.values[0] >= 0.05:
        z_df = group_df[group_df.transform(lambda x: x <= 100)].dropna(how='all')
        plt.figure()
        plt.hist(z_df.loc[:, z_df.columns == sample_colname], color = 'y', alpha=0.5)
        try:
            plt.hist(z_df.loc[:, z_df.columns != sample_colname].iloc[:, 1], alpha=0.5)
        except ValueError:
            continue
        plt.xlabel(sample_colname.split('_')[0])
        plt.ylabel('counts')
        plt.title(cci.stem)
        plt.text(0, 10, 'z score: '+str(z.values[0]))
        print(str(new_root.joinpath(cci.stem+'_'+str(col)+'_100um_hist.png')))
        plt.savefig(str(new_root.joinpath(cci.stem+'_'+str(col)+'_100um_hist.png')))

,"Astro,Chandelier_realdata","Astro,Chandelier_iter1","Astro,Chandelier_iter2","Astro,Chandelier_iter3","Astro,Chandelier_iter4","Astro,Chandelier_iter5","Astro,Chandelier_iter6","Astro,Chandelier_iter7","Astro,Chandelier_iter8","Astro,Chandelier_iter9","Astro,Chandelier_iter10"
149402,12.868789,12.345873,13.101071,12.915605,4.521727,9.571922,13.938663,NaN,NaN,4.582333,NaN
361513,11.381445,11.463780,NaN,1.418219,NaN,9.173022,13.377847,6.440611,7.341837,NaN,13.811766
393975,NaN,NaN,NaN,NaN,NaN,NaN,12.892835,NaN,NaN,NaN,NaN
451207,14.654646,8.587912,NaN,10.532253,14.836392,NaN,NaN,NaN,4.979866,NaN,14.991224
490279,NaN,14.789214,NaN,10.599760,NaN,NaN,NaN,NaN,NaN,NaN,NaN


-4.274204930939026


,"Astro,Chandelier_realdata","Astro,Chandelier_iter1","Astro,Chandelier_iter2","Astro,Chandelier_iter3","Astro,Chandelier_iter4","Astro,Chandelier_iter5","Astro,Chandelier_iter6","Astro,Chandelier_iter7","Astro,Chandelier_iter8","Astro,Chandelier_iter9","Astro,Chandelier_iter10"
149402,12.868789,10.153312,9.341355,NaN,7.364657,NaN,8.030698,NaN,9.054164,13.165294,NaN
158094,NaN,13.454048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
361513,11.381445,6.458811,13.387135,NaN,7.587963,13.489385,14.858804,2.163472,11.430047,7.803587,NaN
393975,NaN,NaN,13.278898,NaN,NaN,8.281174,NaN,NaN,14.160985,NaN,NaN
451207,14.654646,NaN,14.279218,13.940544,NaN,NaN,8.293746,NaN,NaN,NaN,NaN


-4.313960339616392


### test/scratch below

In [ ]:
#do a hist of each column and plot. Can use that output to make sure that this is normal and not all nans
for x in container_df.columns:
    plt.figure()
    plt.hist([container_df.loc[:,x]], bins = 100)
    lab_split = x.split(',')
    plt.xlabel(str(x))
    plt.ylabel('pairwise dist vals')

cell-cell interactions arising from somatic contact or paracrine signaling, which can be inferred based on contact or proximity between cells that occurred with higher frequency than by random chance (29, 30). 

We examined whether these potential cell-cell interactions were cell-type specific. To this end, we only considered cell types at the subclass level and determined the frequency with which cell-cell contacts (or proximity) were observed between two subclasses of cells. 

Two neighboring cells were considered contacting (or in proximity) if their centroid distance was <15 μm, which is approximately the size of the soma of a single cell in both human and mouse cortex (31). 

We determined how much this frequency was above random chance and how significant this difference was by comparing the observed contact frequency with the expected contact frequencies from spatial permutations. 

To avoid artifacts arising from the laminar organization of cells and spatial variation of cells density (namely, cell types in the regions of higher cell density or with similar laminar distributions can result in a higher contact frequency by random chance), we designed our spatial permutations to only disrupt the spatial relationship between neighboring cells while still preserving the laminar distribution and local density of each cell type


![image.png](attachment:image.png)
Fig S10: 
(A) Schematic of spatial permutation test that determines the significance of interactions between cell types. Two cells were considered contacting if their nucleus centroids were within 15 μm in the imaging plane, which is approximately the size of the cell body of a single neuron. Contact frequency between any two cell types was determined as the observed frequency. Then, spatial localization of each cell was randomized within a radius of 50 μm, unless otherwise mentioned. Expected contact frequency between any two cell types was determined in each permutation and such permutation was performed 1,000 times to obtain the distribution of expected contact frequencies. The significance of observed contact frequency was calculated using one-tailed z-test and P-values were corrected to FDR (false discovery rate) using Benjamini-Hochberg Procedure. 
(B) Spatial map of L2/3 IT cells in a human MTG slice. (Left) Measured spatial map. (Middle and right) Two example spatial maps after spatial permutations described in (A). 
(C) Cortical depth distributions of L2/3 IT cells (light blue) and other cells (light red) in the human MTG slice shown in (B). (Left) Measured cortical depth distributions. (Middle and right) Cortical depth distributions after two example spatial permutations described in (A).

Summary: 
- For Each Z plane, for each specimen
    - Grab all cells within said plane
    - Find all sets of individual cells that are within 15um of one another (maybe go cell by cell? Is there a faster way?)
    - group by cell type and calculate observed frequency of cell type - cell type interaction
    - randomize spatial location of all cells by 50um and recalculate observed frequency (save two of these)
    - do so 1000 times to get a distribution
    - calculate one-tailed z-test of observed vs randomized. Benjamini-Hochberg goes here?
    - plot spatial map of specific cell types of interest pre randomization and post 2 randomizations (this will probably involve saving two randomizations and creating new h5ad files for them, possibly concatting them together so they can all be cirro doritod)
    - plot histogram of cortical depth distro of celltypes of interest vs all other cells pre and post 2 permutations.